In [211]:
import pandas as pd
import numpy as np
import string
import sys
import re

#text processing, NLP modules
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import *
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

import gensim.downloader
from gensim.models import KeyedVectors
from gensim.models import Word2Vec

import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (15, 10)

/Users/christophmichel/.pyenv/versions/3.8.5/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [212]:
#!pip install python-Levenshtein

/Users/christophmichel/.pyenv/versions/3.8.5/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [213]:
df = pd.read_csv('Data/tweety_topic.csv')


/Users/christophmichel/.pyenv/versions/3.8.5/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Sentiment analysis with VADER

### Cleaning, feature engineering

In [189]:
def clean_again(x):
    x = str(x)
    x = x.lower()

    #remove twitter handlers
    x = re.sub('@[^\s]+','',x)
    #remove hashtags
    x = re.sub('#[^\s]+','',x)
    #remove URLs
    x = re.sub(r'https?:\/\/[A-Za-z0-9\.\/]+', '', x)
    #remove punctuation
    #x = x.replace('[^\w\s]','')
    #remove single characters
    x = re.sub(r'\s+[a-zA-Z0-9]\s+', '', x)
    #substitute multiple spaces with single space
    x = re.sub(r'\s+', ' ', x, flags=re.I)
    #remove linebreaks
    x = re.sub('\n', '', x)
    #remove special characters
    x = ' '.join(re.findall(r'\w+', x))
    #stemming
    #x = ' '.join([stemmer.stem(word) for word in x.split() ])
    #lemmatizing
    #x = ' '.join([lemmatizer.lemmatize(word) for word in x.split() ])
    #removing stop words
    x = ' '.join([word for word in x.split() if word not in stopwords])
    #removing numbers
    #x = re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", x)
    x = x.split()

    return x

In [191]:
import random
text_data = []
for tweet in df_tweety.text:
    tokens = clean_again(tweet)
    if random.random() > .99:
        print(tokens)
        text_data.append(tokens)

['canada', 'alberta', 'plan', 'send', 'covid', 'team', 'hard', 'hit', 'area', 'edmonton', 'calgary', 'indiablooms', 'first', 'portal']
['possible', 'allergic', 'reaction', 'would', 'cause', 'concern', 'people']
['head', 'warning', 'britain', 'anyone', 'history']
['hour', '1st', 'dose', 'vaccine', 'side', 'effect', 'sore', 'arm']
['accusing', 'use', 'british', 'guinea', 'pig', 'belgium', 'minister']
['heart', 'ache', 'yesterday', 'tragedy', 'feel', 'thing']
['worked', 'day', 'yesterday', 'post', '2nd', 'dose', 'day', 'headache', 'arm', 'pain', 'little', 'fatigue', 'emergen']
['mark', 'historic', 'shift', 'direction', 'country', 'fight', 'covid']
['propaganda', 'war', 'begin', 'bashing', 'va']
['sick', 'amp', 'tired', 'ot', 'virologist', 'amp', 'scientific', 'professional', 'going', 'broadcast', 'medium', 'say', 'nothi']
['least', 'people', 'died', 'far']
['risk', 'vaccine', 'doctor', 'louise', 'lagendijk', 'integrative', 'medicine', 'specialist', 'specializing', 'min']
['common', 'aller

In [194]:
from gensim import corpora
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]
import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [197]:
import gensim
NUM_TOPICS = 20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model5.gensim')

In [198]:
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.030*"worry" + 0.030*"secret" + 0.030*"million" + 0.030*"vaccin"')
(1, '0.057*"vaccine" + 0.038*"dose" + 0.020*"second" + 0.020*"need"')
(2, '0.040*"business" + 0.040*"amp" + 0.020*"hire" + 0.020*"invol"')
(3, '0.031*"dose" + 0.031*"administering" + 0.031*"hey" + 0.031*"2nd"')
(4, '0.029*"first" + 0.029*"kullen" + 0.029*"king" + 0.029*"round"')
(5, '0.034*"parosmia" + 0.034*"feeling" + 0.034*"work" + 0.034*"good"')
(6, '0.027*"busines" + 0.027*"destroy" + 0.027*"ford" + 0.027*"name"')
(7, '0.041*"vaccine" + 0.021*"specializing" + 0.021*"specialist" + 0.021*"lagendijk"')
(8, '0.032*"limited" + 0.032*"vaccine" + 0.032*"discover" + 0.032*"variant"')
(9, '0.040*"feel" + 0.021*"seems" + 0.021*"shaping" + 0.021*"war"')
(10, '0.055*"feel" + 0.037*"amp" + 0.037*"arm" + 0.019*"say"')
(11, '0.029*"republic" + 0.029*"nation" + 0.029*"suspend" + 0.029*"clotting"')
(12, '0.043*"used" + 0.022*"potentially" + 0.022*"technology" + 0.022*"rna"')
(13, '0.034*"emergency" + 0.034*"appears" + 0.034*"

In [200]:
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model5.gensim')

In [207]:
#!pip install pyLDAvis

/Users/christophmichel/.pyenv/versions/3.8.5/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [203]:
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

In [205]:
lda3 = gensim.models.ldamodel.LdaModel.load('model5.gensim')
lda_display3 = pyLDAvis.gensim.prepare(lda3, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display3)

/Users/christophmichel/.pyenv/versions/3.8.5/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
